In [1]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import animation, rc
from IPython.display import HTML

In [2]:
gen_images = np.load("../drawing/generated/samples.npy")
gen_latent = np.load("../drawing/generated/latent.npy")


In [3]:
print("images_shape ", gen_images.shape)
print("latent_shape", gen_latent.shape)

images_shape  (10, 50, 16384)
latent_shape (10, 50, 10)


In [4]:
def sigmoid(x):
    return 1/(1 +np.exp(-x))


In [5]:

def init():
    # make an empty frame
    line_mean.set_ydata(np.nan * np.ones(len(line_mean.get_xdata())))
    line_std.set_ydata(np.nan * np.ones(len(line_std.get_xdata())))
    
    image.set_data([[0]*128, [0]*128])
    return image, line_std, line_mean

In [6]:
def animate(i):
    img = sigmoid(gen_images[-1, i, :].reshape((128, 128)))
    latent = gen_latent[:, i, :]
    latent_mean = np.average(latent, axis=1)
    latent_std = np.std(latent, axis=1)
    
    line_mean.set_ydata(latent_mean)
    line_std.set_ydata(latent_std)
    image.set_data(img)
    
    return image, line_std, line_mean

In [11]:
%%capture 

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(15, 10))
image = axs[0].imshow(sigmoid(gen_images[-1, [0]].reshape((128, 128))), cmap="Greys_r")
fig.suptitle("Animation of generated events and their associated latent expressions")

means = np.average(gen_latent[:,0], axis=1)
stds = np.std(gen_latent[:,0], axis=1)
x = np.arange(gen_latent[:,0].shape[0])

line_mean, = axs[1].plot(x, means, "x-", label=r"$\mu$")
line_std, = axs[1].plot(x, stds, "o-", label=r"$\sigma$")

axs[1].legend()
axs[1].set_ylim((-1, 2.0))

anim = animation.FuncAnimation(
    fig,
    animate,
    np.arange(gen_images.shape[1]),
    init_func=init,
    interval=1000
    )


In [12]:
HTML(anim.to_html5_video())